In [1]:
import cv2
import numpy as np
import yaml
import tarfile
import os

In [ ]:
def extract_calibration_data(tar_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        yaml_file = [f for f in tar.getnames() if f.endswith('ost.yaml')][0]
        tar.extract(yaml_file)

        with open(yaml_file, 'r') as f:
            calib_data = yaml.safe_load(f)
            
        camera_matrix = np.array(calib_data['camera_matrix']['data']).reshape(3, 3)
        dist_coeffs = np.array(calib_data['distortion_coefficients']['data'])
        return camera_matrix, dist_coeffs
    
def extract_images(tar_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        image_files = [f for f in tar.getnames() if f.endswith('.png')]
        tar.extractall()
    return image_files

In [6]:
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
# https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html

def calibrate_with_different_models(images_folder):
    """Compare different distortion models"""
    CHECKERBOARD = (8, 6)  # TODO check this
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
    
    objpoints, imgpoints = [], []
    
    for fname in os.listdir(images_folder):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            img = cv2.imread(os.path.join(images_folder, fname))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
            if ret:
                objpoints.append(objp)
                corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)
    
    results = {}
    
    # (k1, k2, p1, p2, k3)
    # k1, k2, k3 - radial distortion coefficients
    # p1, p2 - tangential distortion coefficients
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )
    results['default'] = {'matrix': mtx, 'dist': dist, 'error': ret}
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None,
        flags=cv2.CALIB_RATIONAL_MODEL
    )
    results['rational'] = {'matrix': mtx, 'dist': dist, 'error': ret}
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None,
        flags=cv2.CALIB_THIN_PRISM_MODEL
    )
    results['thin_prism'] = {'matrix': mtx, 'dist': dist, 'error': ret}
    
    def calc_reprojection_error(objpoints, imgpoints, mtx, dist, rvecs, tvecs):
        mean_error = 0
        for i in range(len(objpoints)):
            imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
            error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
            mean_error += error
        return mean_error/len(objpoints)
    
    for model in results:
        error = calc_reprojection_error(
            objpoints, imgpoints, 
            results[model]['matrix'], 
            results[model]['dist'],
            rvecs, tvecs
        )
        results[model]['reprojection_error'] = error
    
    return results

In [5]:
camera_matrix, dist_coeffs = extract_calibration_data('calibrationdata.tar.gz')
print("camera matrix and distortion coefficients extracted from calibration.tar.gz:")
print(camera_matrix)
print("_"*50)
print(dist_coeffs)

camera matrix and distortion coefficients extracted from calibration.tar.gz:
[[290.46301   0.      312.90291]
 [  0.      290.3703  203.01488]
 [  0.        0.        1.     ]]
__________________________________________________
[-2.79797e-01  6.43090e-02 -6.80000e-05  1.96700e-03  0.00000e+00]


In [7]:
results = calibrate_with_different_models('calibration_images/')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'calibration_images/'

In [ ]:
def compare_distortion(dist1, dist2):
    """Compare two distortion coefficients"""
    print("Default distortion:", dist1)
    print("Advanced distortion:", dist2)
    print("Difference:", np.abs(dist1 - dist2))